# cbvEval
일단 LamCalc 노트북에서 정의했던 람다식 문법 데이타 타입과 출력을 위한 문자열 변환함수 등을 그대로 가져오자.

In [1]:
-- 변수 이름은 문자열 나타낸다
type Nm = String

-- 람다식 문법 구조
data Tm = Var Nm | Lam Nm Tm | App Tm Tm  deriving (Show, Eq)

-- 람다식을 보기좋게 문자열로 변환해주는 함수
ppTm (Var x) = x
ppTm (Lam x t) = "\\" ++ x ++ " -> " ++ ppTm t
ppTm (App t s) = ppt t ++ " " ++ pps s
  where
  ppt t@(Lam{}) = paren (ppTm t)
  ppt t         = ppTm t
  pps s@(Var{}) = ppTm s
  pps s         = paren (ppTm s)

paren s = "(" ++ s ++ ")"

In [2]:
idTm = Lam "x" (Var "x")
ttTm = Lam "x" (Lam "y" (Var "x")) 
ffTm = Lam "x" (Lam "y" (Var "y")) 

putStr . ppTm $ Var "x"
putStr . ppTm $ idTm
putStr . ppTm $ App idTm idTm
putStr . ppTm $ App (App ttTm (Var "u")) (Var "w")

x

\x -> x

(\x -> x) (\x -> x)

(\x -> \y -> x) u w

In [3]:
-- 하스켈에서 그대로 붙여넣으면 실행도 어느 정도 가능하게
let z = "z" in (\x -> x) ((\x -> x) z)

Line 1: Use id
Found:
\ x -> x
Why not:
idLine 1: Use id
Found:
\ x -> x
Why not:
id

"z"

In [4]:
-- 좀더 예쁘게 유니코드의 그리스 문자 람다 등과 색상 등을 이용해 보여주는 함수들
import IHaskell.Display

dispTm t = Display [html("<code>"++dpTm t++"</code>")]

dpTm (Var x) = x
dpTm (Lam x t) = "λ" ++ x ++ "." ++ dpTm t
dpTm (App t s) = dpt t ++ " " ++ dps s
  where
  dpt t@(Lam{}) = paren (dpTm t)
  dpt t         = dpTm t
  dps s@(Var{}) = dpTm s
  dps s         = paren (dpTm s)

dispTm $ Lam "y" (App idTm (App idTm (Var "y")))

λy.(λx.x) ((λx.x) y)

In [5]:
putStrLn $ dpTm $ Lam "y" (App idTm (App idTm (Var "y")))

λy.(λx.x) ((λx.x) y)

# 람다식의 call-by-value evaluation
call-by-value evaluation (줄여서 CVB evaluation)은
적극적 계산법(eager evaluation)이라고 부르기도 한다.

람다식의 적극적 계산법을 아래와 같은 표기법을 써서 정의할 수 있다.

$\displaystyle\begin{array}{rcrcl}
\sigma&\in&\textit{Env} &=&
\textit{Nm} \longrightarrow^{\hspace{-2.7ex}\textrm{fin}} \textit{Val} \\
(\lambda x.t,\sigma)&\in&\textit{Val} &=& \textit{Tm}_V \times Env
\end{array}$

$\displaystyle\begin{array}{rcl}
\textit{Tm}_V &\subset& Tm\\
\textit{Tm}_V &=& \{ \lambda x.t \mid x\in\textit{Nm}, t\in\textit{Tm}\}
\end{array}$

$~$

$\displaystyle\Downarrow ~:~ Tm\times Env \to Val $

$\displaystyle\frac{}{(x, \sigma) \Downarrow \sigma(x)} 
\quad
 \displaystyle\frac{}{(\lambda x.t, \sigma) \Downarrow \langle\lambda x.t, \sigma\rangle}
\quad
 \displaystyle\frac{~
   \begin{array}{l}
   (t_1,\sigma) \Downarrow \langle\lambda x.t, \sigma_1\rangle \\
   (t_2,\sigma) \Downarrow v_2 \\
   (t, \{x\mapsto v_2\}\sigma_1) \Downarrow v
   \end{array}~}{
   (t_1~t_2, \sigma) \Downarrow v}$

위의 표기법은 비유하자면 $y = f(x)$라고 쓰는 대신에 $f(x)$를 기계적인 규칙을 따라 계속 계산을 돌리다 보면 $y$라는 값에 도달한다는 의미로 $f(x) \Downarrow v$라는 식으로 표시하는 것이다. 그리고 바로 람다식의 적극적 계산법을 위한 기계적인 규칙이 위에 나와 있는 세 가지 규칙이다.

In [6]:
-- finite mapping을 순서쌍 리스트 타입으로 정의
type Env = [(Nm, Val)]
-- Tm_V를 따로 정의하지 않고 그냥 Tm을 이용해 정의
data Val = Cl Tm Env  deriving Show

-- simga(x)에 해당하는 것이 lookup x simga
lookup' x env  = v  where Just v = lookup x env

eval :: Tm -> Env -> Val
eval v@(Lam x t) env = Cl v env
eval (Var x)     env = lookup' x env
eval (App t1 t2) env = eval t ((x,v2):env1)
  where
    Cl (Lam x t) env1 = eval t1 env
    v2                = eval t2 env

:type eval
:type uncurry eval

eval (App (Lam "x" (Var "x")) (Lam "x" (Var "x"))) []

eval :: Tm -> Env -> Val

uncurry eval :: (Tm, Env) -> Val

Cl (Lam "x" (Var "x")) []